In [ ]:
# Setup the Jupyter version of Dash
from dash import Dash


# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "jacobgriffin2133"
password = "Tateisnice30-"

# Connect to database via CRUD Module
db = AnimalShelter()
print("connection successful") #I created this statement so I knew the connection was successful

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))


# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

## Debug
#print(len(df.to_dict(orient='records')))
#print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = Dash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
# image_filename = 'GraziosoSalvareLogo.png' # replace with your own image
# encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date

app.layout = html.Div([
    # html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Jacob Griffin CS-340 Dashboard'))), #unique identifier
    html.Hr(),
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        
        #Radio Buttons created for filtering options
        #and gives the user the ability to select
        #these options using the radio buttons
        dcc.RadioItems(
            id='filter-type',
            options=[
            {'label': 'Water Rescue', 'value': 'WR'},
            {'label': 'Mountain Rescue', 'value': 'MR'},
            {'label': 'Disaster Rescue', 'value': 'DR'},
            {'label': 'Reset to Unfiltered breed type', 'value': 'RESET'}
            ],
            value='RESET', #Default selection when the dashboard is initially opened
            labelStyle={'display': 'inline-block'} #Displays the labels "in-line" with each other
        )
    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
                         
        row_selectable="single", #using single row selection
        selected_rows=[0], #selects the first row by default
        page_size = 10, #pagination to enhance the user experience
        sort_action = 'native', #gives the user the ability to sort the page
        filter_action = 'native', #gives the user the ability to filter the page
        
                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
## FIX ME Add code to filter interactive data table with MongoDB queries

    #Created if and elif statements to cypher through the database
    #and read the data and filter it in accordance with the selected
    #radio button that we previously established
    #for the age we used the $gte (greater than or equal to) and $lte (less than or equal to) methods

        #Filter for the Water Rescue breeds
        if filter_type == 'WR':
            df = pd.DataFrame(list(db.read({'$and': [{'sex_upon_outcome': 'Intact Female'},
                                                          {'$or': [
                                                              {'breed': 'Labrador Retriever Mix'},
                                                              {'breed': 'Chesapeke Bay Retriever'},
                                                              {'breed': 'Newfoundland'}]
                                                          },
                                                          {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                                    {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                                          }]
                                                })))
        #filter for the Mountain/Wilderness breeds
        elif filter_type == 'MR':
            df = pd.DataFrame(list(db.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                          {'$or': [
                                                              {'breed': 'German Shepherd'},
                                                              {'breed': 'Alaskan Malamute'},
                                                              {'breed': 'Old English Sheepdog'},
                                                              {'breed': 'Siberian Husky'},
                                                              {'breed': 'Rottweiler'}]
                                                          },
                                                          {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                                    {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                                          }]
                                                })))
        #filter for the Disaster/Individual Tracking breeds
        elif filter_type == 'DR':
            df = pd.DataFrame(list(db.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                          {'$or': [
                                                              {'breed': 'Doberman Pinscher'},
                                                              {'breed': 'German Shepherd'},
                                                              {'breed': 'Golden Retriever'},
                                                              {'breed': 'Bloodhound'},
                                                              {'breed': 'Rottweiler'}]
                                                          },
                                                          {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 20}},
                                                                    {'age_upon_outcome_in_weeks': {'$lte': 300}}]
                                                          }]
                                                })))
        #Reset the data table on the dashboard 
        elif filter_type == 'RESET':
            df = pd.DataFrame.from_records(db.read({}))


        #I could not return with columns without getting an error
        #so I had to just return the Data to get the filtering to work
        if '_id' in df.columns:
            df.drop(columns=['_id'], inplace=True)
        data=df.to_dict('records')

        return data

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None:
        return []

    dff = pd.DataFrame.from_dict(viewData)
    
    #Count the number of times the breed is read 
    #in the database
    breed_counts = dff['breed'].value_counts()
    
    #Creation of the Pie chart and filtering
    #when selecting different Rescue Types
    #This will update the chart with the correct numerical percentages
    #and display them accordingly
    fig = px.pie(
        data_frame=dff,
        names=breed_counts.index,
        values=breed_counts.values,
        color_discrete_sequence=px.colors.sequential.RdBu,
        width=800,
        height=500,
    )
    
    return [
        dcc.Graph(
            figure=fig
        )
    ]
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns:
        return{}
    
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(mode='inline', debug=True)


Connection to database Successful
connection successful
Read Complete


[2025-04-06 14:59:02,658] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\jacob\AppData\Local\Programs\Python\Python313\Lib\site-packages\flask\app.py", line 880, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\jacob\AppData\Local\Programs\Python\Python313\Lib\site-packages\flask\app.py", line 865, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^
  File "C:\Users\jacob\AppData\Local\Programs\Python\Python313\Lib\site-packages\dash\dash.py", line 1376, in dispatch
    ctx.run(
    ~~~~~~~^
        functools.partial(
        ^^^^^^^^^^^^^^^^^^
    ...<7 lines>...
        )
        ^
    )
    ^
  File "C:\Users\jacob\AppData\Local\Programs\Python\Python313\Lib\site-packages\dash\_callback.py", line 507, in add_context
    raise err
  File "C:\User

Read Complete
